In [80]:
import os
import requests
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq

In [81]:
# CONSTANTES
CLIENT_ID = os.getenv("YELP_CLIENT_ID")
YELP_API_KEY = os.getenv("YELP_API_KEY")
GCP_PROJECT = os.environ.get("GCP_PROJECT")
TABLE_ID = "businesses"
DATASET_ID = "astoria"

In [82]:
# Realizar la solicitud HTTP
def request_yelp(api_key, table_id, params):    
    url = f"https://api.yelp.com/v3/{table_id}/search?{params}"
    headers = {
        "accept": "application/json",
        "Authorization": "Bearer " + api_key
    }
    response = requests.get(url, headers=headers)
    return response.json()

In [106]:
# Normalizar la respuesta JSON y convertirla en un objeto pyarrow.Table
def normalize(json_response, table_id):
    # seleccionar la columna "businesses"
    businesses = json_response[table_id]    
    # Convertir el objeto JSON en un DataFrame de pandas
    df = pd.json_normalize(businesses)
    # Renombrar las columnas
    df = df.rename(columns={"id": "business_id"})
    df = df.rename(columns={"rating": "stars"})
    df = df.rename(columns={"coordinates.latitude": "latitude"})
    df = df.rename(columns={"coordinates.longitude": "longitude"})
    df = df.rename(columns={"location.address1": "address1"})
    df = df.rename(columns={"location.address2": "address2"})
    df = df.rename(columns={"location.address3": "address3"})
    df = df.rename(columns={"location.city": "city"})
    df = df.rename(columns={"location.zip_code": "postal_code"})
    df = df.rename(columns={"location.country": "country"})
    df = df.rename(columns={"location.state": "state"})
    df = df.rename(columns={"location.display_address": "address"})    
    # Convertir is_closed a is_open invirtiendo el valor    
    df["is_closed"] = df["is_closed"].apply(lambda x: 0 if x else 1)
    df = df.rename(columns={"is_closed": "is_open"})
    # Extraer valor de title de categories y guardar un string separado por comas
    df["categories"] = df["categories"].apply(lambda x: ", ".join([y["title"] for y in x]))
    # Convertir address en string separado por espacios
    df["address"] = df["address"].apply(lambda x: " ".join(x))
    # Crear columna date #TODO: obtener de check-in
    df["date"] = "2013-01-05 14:52:30, 2013-01-19 14:21:37, 2013-04-10 01:18:43, 2013-04-13 14:40:13, 2013-10-02 23:06:36, 2013-10-30 22:13:59, 2013-11-06 23:58:42, 2015-08-14 22:12:45"
    # Seleccionar las columnas que se van a usar
    fields = ["business_id", "date", "name", "address", "city", "state", "postal_code", "latitude", "longitude", "stars", "review_count", "is_open", "categories"]
    df = df[fields]
    # df = df[df["categories"].str.contains(category, case=False, na=False)]
    return df

In [107]:
# Selecciono la categoría en Yelp que seo buscar y analizar
category = "recreation"
# Cargar la respuesta JSON en un objeto Python
params = f"categories={category}&location=FL&sort_by=best_match"
data = request_yelp(YELP_API_KEY, TABLE_ID, params)
df = normalize(data, TABLE_ID)
df.columns

Index(['business_id', 'date', 'name', 'address', 'city', 'state',
       'postal_code', 'latitude', 'longitude', 'stars', 'review_count',
       'is_open', 'categories'],
      dtype='object')

In [108]:
# Convertir el DataFrame de pandas a un objeto pyarrow.Table
df = pa.Table.from_pandas(df)
# Guardar el objeto pyarrow.Table en formato Parquet
pq.write_table(df, TABLE_ID+".parquet")

In [109]:
# Leer el DataFrame desde el archivo Parquet
df = pd.read_parquet(TABLE_ID+".parquet")
df.head()

,business_id,date,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
0,7kk7PZvEeeb9fg0XgpIf1g,"2013-01-05 14:52:30, 2013-01-19 14:21:37, 2013...",Patti and Allan Herbert Wellness Center,"1241 Dickinson Dr Coral Gables, FL 33146",Coral Gables,FL,33146,25.715676,-80.280477,4.5,31,1,Recreation Centers
1,jl3sNS0Yg2-LN32O8H6CSg,"2013-01-05 14:52:30, 2013-01-19 14:21:37, 2013...",Hit Zone Inc,"7900 SW 40th St Miami, FL 33155",Miami,FL,33155,25.733021,-80.323534,4.0,17,1,Recreation Centers
2,t3aYNpAvVxKT-CfU36kZyw,"2013-01-05 14:52:30, 2013-01-19 14:21:37, 2013...",Florida International University Recreation Ce...,"11290 SW 12th St WRC 101 Miami, FL 33199",Miami,FL,33199,25.756015,-80.378090,3.5,14,1,"Trainers, Recreation Centers"
3,obhS1T1ZlakTmR1Ke8Luag,"2013-01-05 14:52:30, 2013-01-19 14:21:37, 2013...",Coral Gables War Memorial Youth Center,"405 University Dr Coral Gables, FL 33134",Coral Gables,FL,33134,25.741908,-80.263279,3.5,13,1,"Parks, Playgrounds, Recreation Centers"
4,cCLAEzDoPBzsHePs_23Kbw,"2013-01-05 14:52:30, 2013-01-19 14:21:37, 2013...",Kids Empire Miami Dolphin Mall,"11401 NW 12th St Ste E610 Miami, FL 33172",Miami,FL,33172,25.787996,-80.380240,3.0,23,1,"Recreation Centers, Indoor Playcentre, Venues ..."


In [110]:
# Guardar el dataframe en BigQuery
def df_to_bq(df, destination_table, project_id):
    """
    Guarda un dataframe de pandas en BigQuery
    """
    # convertir a string
    df = df.astype(str)
    # guardar en BigQuery
    df.to_gbq(destination_table=destination_table, project_id=project_id, if_exists="replace")

In [111]:
# tabla de destino
destination_table = f"{DATASET_ID}.yelp-{TABLE_ID}"
df_to_bq(df, destination_table, GCP_PROJECT)